# BERT
Here an example of training and evaluation for the BertForSequenceClassification.

## Training

In [ ]:
from transformers import  MobileBertConfig,MobileBertForSequenceClassification, DistilBertConfig,BertConfig, BertForSequenceClassification, AutoTokenizer, AdamW, DistilBertForSequenceClassification
import torch
import json
import random

data = []
epochs = 30 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
with open('/content/train.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))

# Preprocess data
sentences = []
labels = []
for item in data:
    for option, answer in zip([item['option1'], item['option2']], [item['answer'], item['answer']]):
        sentence = item['sentence'].replace('_', '[MASK]')
        sentence = sentence.replace('[MASK]', option)
        sentences.append(sentence)
        labels.append(int(answer) - 1)

# Split data into training and dev sets
split_ratio = 0.8  
split_index = int(len(sentences) * split_ratio)

train_sentences = sentences[:split_index]
train_labels = torch.tensor(labels[:split_index]).to(device)

dev_sentences = sentences[split_index:]
dev_labels = torch.tensor(labels[split_index:]).to(device)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

config = BertConfig.from_pretrained("bert-base-uncased", num_labels=2)
model = BertForSequenceClassification(config)
model = model.to(device)



optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-4)
#optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

# Early stopping parameters
early_stopping_patience = 5
best_train_loss = float('inf')
early_stopping_counter = 0

batch_size = 128
print("------------------------------------")
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train_loss = 0.0
    # Shuffle training data each epoch
    combined_data = list(zip(train_sentences, train_labels))
    random.shuffle(combined_data)
    train_sentences, train_labels = zip(*combined_data)
    train_labels = torch.tensor(train_labels).to(device)

    # Training phase
    model.train()
    for i in range(0, len(train_sentences), batch_size):
        batch_train_sentences = train_sentences[i:i + batch_size]
        batch_train_labels = train_labels[i:i + batch_size]
        batch_train_labels = batch_train_labels.to(device)

        optimizer.zero_grad()
        batch_train_inputs = tokenizer(batch_train_sentences, return_tensors="pt", padding=True, truncation=True)
        batch_train_inputs = {key: value.to(device) for key, value in batch_train_inputs.items()}
        outputs = model(**batch_train_inputs, labels=batch_train_labels)
        loss = outputs.loss
        train_loss += loss
        loss.backward()
        optimizer.step()
    avg_train_loss = train_loss / (len(train_labels)/batch_size)
    print(f"Training Loss: {avg_train_loss}")

    # Validation phase
    model.eval()
    with torch.no_grad():
        dev_loss = 0.0
        for i in range(0, len(dev_labels), batch_size):
            batch_dev_sentences = dev_sentences[i:i + batch_size]
            batch_dev_labels = dev_labels[i:i + batch_size]

            batch_dev_labels = batch_dev_labels.to(device)
            batch_dev_inputs = tokenizer(batch_dev_sentences, return_tensors="pt", padding=True, truncation=True)
            batch_dev_inputs = {key: value.to(device) for key, value in batch_dev_inputs.items()}

            outputs = model(**batch_dev_inputs, labels=batch_dev_labels)
            dev_loss += outputs.loss.item()

        avg_dev_loss = dev_loss / (len(dev_labels) / batch_size)
        print(f"Validation Loss: {avg_dev_loss}")
        print("------------------------------------")

    # Early stopping logic
    if avg_dev_loss < best_train_loss:
        best_train_loss = avg_dev_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= early_stopping_patience:
        print(f"Early stopping! No improvement in validation loss for {early_stopping_patience} epochs.")
        break 

# Save trained model
model.save_pretrained('/content/bertmodel')

print("Training complete.")






## Test

In [ ]:
from transformers import BertForSequenceClassification, AdamW, AutoTokenizer
import torch
import json
from sklearn.metrics import accuracy_score

# Load test dataset
test_data = []
with open('/content/test.jsonl', 'r') as f:
    for line in f:
        test_data.append(json.loads(line))

# Preprocess test data
test_sentences = []
test_labels = []
for item in test_data:
    for option, answer in zip([item['option1'], item['option2']], [item['answer'], item['answer']]):
        sentence = item['sentence'].replace('_', '[MASK]')
        sentence = sentence.replace('[MASK]', option)
        test_sentences.append(sentence)
        test_labels.append(int(answer) - 1)

# Load trained model
loaded_model = BertForSequenceClassification.from_pretrained('/content/bertmodel')
loaded_model.to(device)

# Tokenize  test data
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
test_inputs = tokenizer(test_sentences, return_tensors="pt", padding=True, truncation=True)
test_inputs = {key: value.to(device) for key, value in test_inputs.items()}

# Inference on test data
loaded_model.eval()
with torch.no_grad():
    outputs = loaded_model(**test_inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    predicted_labels = torch.argmax(probabilities, dim=1).cpu().numpy()

true_labels = torch.tensor(test_labels).numpy()

# Print sentences, correct answers, predicted labels
for i in range(len(test_data)):
    print(f"Example {i + 1}:")
    print("Sentence:", test_sentences[i])
    print("Correct Answer:", true_labels[i] + 1)
    print("Predicted Label:", predicted_labels[i] + 1)
    print("--------------")

# Evaluate model performance
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy on the test set: {accuracy}")

# DistilBERT
Here an example of training and evaluation of DistilBertForSequenceClassification.

## Training

In [ ]:
from transformers import  DistilBertConfig, AutoTokenizer, AdamW, DistilBertForSequenceClassification
import torch
import json
import random

data = []
epochs = 30 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
with open('/content/train.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))

# Preprocess data
sentences = []
labels = []
for item in data:
    for option, answer in zip([item['option1'], item['option2']], [item['answer'], item['answer']]):
        sentence = item['sentence'].replace('_', '[MASK]')
        sentence = sentence.replace('[MASK]', option)
        sentences.append(sentence)
        labels.append(int(answer) - 1)

# Split data into training and dev sets
split_ratio = 0.8  
split_index = int(len(sentences) * split_ratio)

train_sentences = sentences[:split_index]
train_labels = torch.tensor(labels[:split_index]).to(device)

dev_sentences = sentences[split_index:]
dev_labels = torch.tensor(labels[split_index:]).to(device)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

dropout_rate = 0.3
config = DistilBertConfig.from_pretrained("distilbert-base-uncased", num_labels=2, dropout=dropout_rate)
model = DistilBertForSequenceClassification(config)
model = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

# Early stopping parameters
early_stopping_patience = 5
best_train_loss = float('inf')
early_stopping_counter = 0

batch_size = 128
print("------------------------------------")
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train_loss = 0.0
    # Shuffle training data each epoch
    combined_data = list(zip(train_sentences, train_labels))
    random.shuffle(combined_data)
    train_sentences, train_labels = zip(*combined_data)
    train_labels = torch.tensor(train_labels).to(device)

    # Training phase
    model.train()
    for i in range(0, len(train_sentences), batch_size):
        batch_train_sentences = train_sentences[i:i + batch_size]
        batch_train_labels = train_labels[i:i + batch_size]
        batch_train_labels = batch_train_labels.to(device)

        optimizer.zero_grad()

        batch_train_inputs = tokenizer(batch_train_sentences, return_tensors="pt", padding=True, truncation=True, return_token_type_ids=False)
        batch_train_inputs = {key: value.to(device) for key, value in batch_train_inputs.items()}
        outputs = model(**batch_train_inputs, labels=batch_train_labels)
        loss = outputs.loss
        train_loss += loss
        loss.backward()
        optimizer.step()
    avg_train_loss = train_loss / (len(train_labels)/batch_size)
    print(f"Training Loss: {avg_train_loss}")

    # Validation phase
    model.eval()
    with torch.no_grad():
        dev_loss = 0.0
        for i in range(0, len(dev_labels), batch_size):
            batch_dev_sentences = dev_sentences[i:i + batch_size]
            batch_dev_labels = dev_labels[i:i + batch_size]
            batch_dev_labels = batch_dev_labels.to(device)
            batch_dev_inputs = tokenizer(batch_dev_sentences, return_tensors="pt", padding=True, truncation=True, return_token_type_ids=False)
            batch_dev_inputs = {key: value.to(device) for key, value in batch_dev_inputs.items()}
            outputs = model(**batch_dev_inputs, labels=batch_dev_labels)
            dev_loss += outputs.loss.item()

        avg_dev_loss = dev_loss / (len(dev_labels) / batch_size)
        print(f"Validation Loss: {avg_dev_loss}")
        print("------------------------------------")

    # Early stopping logic
    if avg_dev_loss < best_train_loss:
        best_train_loss = avg_dev_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= early_stopping_patience:
        print(f"Early stopping! No improvement in validation loss for {early_stopping_patience} epochs.")
        break  

# Save trained model
model.save_pretrained('/content/model')

print("Training complete.")

## Test

In [ ]:
from transformers import DistilBertForSequenceClassification, AdamW, AutoTokenizer
import torch
import json
from sklearn.metrics import accuracy_score

# Load test dataset
test_data = []
with open('/content/test.jsonl', 'r') as f:
    for line in f:
        test_data.append(json.loads(line))

# Preprocess test data
test_sentences = []
test_labels = []
for item in test_data:
    for option, answer in zip([item['option1'], item['option2']], [item['answer'], item['answer']]):
        sentence = item['sentence'].replace('_', '[MASK]')
        sentence = sentence.replace('[MASK]', option)
        test_sentences.append(sentence)
        test_labels.append(int(answer) - 1)

# Load trained model
loaded_model = DistilBertForSequenceClassification.from_pretrained('/content/distilbertmodel')
loaded_model.to(device)

# Tokenize  test data
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
test_inputs = tokenizer(test_sentences, return_tensors="pt", padding=True, truncation=True)
test_inputs = {key: value.to(device) for key, value in test_inputs.items()}

# Inference on test data
loaded_model.eval()
with torch.no_grad():
    outputs = loaded_model(**test_inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    predicted_labels = torch.argmax(probabilities, dim=1).cpu().numpy()

true_labels = torch.tensor(test_labels).numpy()

# Print sentences, correct answers, predicted labels
for i in range(len(test_data)):
    print(f"Example {i + 1}:")
    print("Sentence:", test_sentences[i])
    print("Correct Answer:", true_labels[i] + 1)
    print("Predicted Label:", predicted_labels[i] + 1)
    print("--------------")

# Evaluate model performance
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy on the test set: {accuracy}")

# MobileBERT
Here an example of training and evaluation of MobileBertForSequenceClassification.

## Training

In [ ]:
from transformers import  MobileBertConfig, AutoTokenizer, AdamW, MobileBertForSequenceClassification
import torch
import json
import random

data = []
epochs = 30r
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
with open('/content/train.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))

# Preprocess data
sentences = []
labels = []
for item in data:
    for option, answer in zip([item['option1'], item['option2']], [item['answer'], item['answer']]):
        sentence = item['sentence'].replace('_', '[MASK]')
        sentence = sentence.replace('[MASK]', option)
        sentences.append(sentence)
        labels.append(int(answer) - 1)

# Split data into training and dev sets
split_ratio = 0.8
split_index = int(len(sentences) * split_ratio)

train_sentences = sentences[:split_index]
train_labels = torch.tensor(labels[:split_index]).to(device)

dev_sentences = sentences[split_index:]
dev_labels = torch.tensor(labels[split_index:]).to(device)

tokenizer = AutoTokenizer.from_pretrained("lordtt13/emo-mobilebert")

config = MobileBertConfig.from_pretrained("lordtt13/emo-mobilebert", num_labels=2)
model = MobileBertForSequenceClassification(config)
model = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

# Early stopping parameters
early_stopping_patience = 5
best_train_loss = float('inf')
early_stopping_counter = 0

batch_size = 128
print("------------------------------------")
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train_loss = 0.0
    # Shuffle training data each epoch
    combined_data = list(zip(train_sentences, train_labels))
    random.shuffle(combined_data)
    train_sentences, train_labels = zip(*combined_data)
    train_labels = torch.tensor(train_labels).to(device)

    # Training phase
    model.train()
    for i in range(0, len(train_sentences), batch_size):
        batch_train_sentences = train_sentences[i:i + batch_size]
        batch_train_labels = train_labels[i:i + batch_size]
        batch_train_labels = batch_train_labels.to(device)

        optimizer.zero_grad()

        batch_train_inputs = tokenizer(batch_train_sentences, return_tensors="pt", padding=True, truncation=True, return_token_type_ids=False)
        batch_train_inputs = {key: value.to(device) for key, value in batch_train_inputs.items()}
        outputs = model(**batch_train_inputs, labels=batch_train_labels)
        loss = outputs.loss
        train_loss += loss
        loss.backward()
        optimizer.step()
    avg_train_loss = train_loss / (len(train_labels)/batch_size)
    print(f"Training Loss: {avg_train_loss}")

    # Validation phase
    model.eval()
    with torch.no_grad():
        dev_loss = 0.0
        for i in range(0, len(dev_labels), batch_size):
            batch_dev_sentences = dev_sentences[i:i + batch_size]
            batch_dev_labels = dev_labels[i:i + batch_size]
            batch_dev_labels = batch_dev_labels.to(device)
            batch_dev_inputs = tokenizer(batch_dev_sentences, return_tensors="pt", padding=True, truncation=True, return_token_type_ids=False)
            batch_dev_inputs = {key: value.to(device) for key, value in batch_dev_inputs.items()}
            outputs = model(**batch_dev_inputs, labels=batch_dev_labels)
            dev_loss += outputs.loss.item()

        avg_dev_loss = dev_loss / (len(dev_labels) / batch_size)
        print(f"Validation Loss: {avg_dev_loss}")
        print("------------------------------------")

    # Early stopping logic
    if avg_dev_loss < best_train_loss:
        best_train_loss = avg_dev_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= early_stopping_patience:
        print(f"Early stopping! No improvement in validation loss for {early_stopping_patience} epochs.")
        break 

# Save trained model
model.save_pretrained('/content/mobilebertmodel')

print("Training complete.")

## Test

In [ ]:
from transformers import MobileBertForSequenceClassification, AdamW, AutoTokenizer
import torch
import json
from sklearn.metrics import accuracy_score

# Load test dataset
test_data = []
with open('/content/test.jsonl', 'r') as f:
    for line in f:
        test_data.append(json.loads(line))

# Preprocess test data
test_sentences = []
test_labels = []
for item in test_data:
    for option, answer in zip([item['option1'], item['option2']], [item['answer'], item['answer']]):
        sentence = item['sentence'].replace('_', '[MASK]')
        sentence = sentence.replace('[MASK]', option)
        test_sentences.append(sentence)
        test_labels.append(int(answer) - 1)

# Load trained model
loaded_model = MobileBertForSequenceClassification.from_pretrained('/content/mobilebertmodel')
loaded_model.to(device)

# Tokenize  test data
tokenizer = AutoTokenizer.from_pretrained("lordtt13/emo-mobilebert")
test_inputs = tokenizer(test_sentences, return_tensors="pt", padding=True, truncation=True)
test_inputs = {key: value.to(device) for key, value in test_inputs.items()}

# Inference on test data
loaded_model.eval()
with torch.no_grad():
    outputs = loaded_model(**test_inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    predicted_labels = torch.argmax(probabilities, dim=1).cpu().numpy()

true_labels = torch.tensor(test_labels).numpy()

# Print sentences, correct answers, and predicted labels
for i in range(len(test_data)):
    print(f"Example {i + 1}:")
    print("Sentence:", test_sentences[i])
    print("Correct Answer:", true_labels[i] + 1)
    print("Predicted Label:", predicted_labels[i] + 1)
    print("--------------")

# Evaluate model performance
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy on the test set: {accuracy}")